<a href="https://colab.research.google.com/github/Ravjot-anand/ML-Algorithms/blob/main/Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [3]:
X,y = make_classification(n_samples=10000, n_features=10,n_informative=3)

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.9625


# Bagging


In [6]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
)

In [7]:
bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [8]:
y_pred = bag.predict(X_test)


accuracy_score(y_test,y_pred)

0.9775

In [9]:
bag.estimators_samples_[0].shape


(4000,)

In [10]:
bag.estimators_features_[0].shape


(10,)

# Pasting

In [11]:

bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False, #no repeat
    random_state=42,
    verbose = 1,
    n_jobs=-1
)

In [12]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(y_test,y_pred))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   15.5s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Pasting classifier 0.971


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.3s finished


# Random Subspaces

In [13]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False, #no repaeat
    max_features=0.5,
    bootstrap_features=True, #but  features are drawn with replacement.
    random_state=42
)

In [14]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))

Random Subspaces classifier 0.9645


# Random Patches

In [15]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [16]:

bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))



Random Patches classifier 0.9575


# OOB score

In [17]:
#There are some rows or sample which dont come out and statistically there are 35% rows

bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

In [18]:
bag.fit(X_train,y_train)


BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [19]:
#score of that rows seperatly
bag.oob_score_


0.97325

In [20]:
#Total score
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))


Accuracy 0.9705


In [21]:
'''
Bagging Tips
Bagging generally gives better results than Pasting
Good results come around the 25% to 50% row sampling mark
Random patches and subspaces should be used while dealing with high dimensional data
To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV
'''

'\nBagging Tips\nBagging generally gives better results than Pasting\nGood results come around the 25% to 50% row sampling mark\nRandom patches and subspaces should be used while dealing with high dimensional data\nTo find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV\n'

**Applying GridSearchCV**

In [22]:
#We can find hyperparameters with gridsearchcv

In [23]:
from sklearn.model_selection import GridSearchCV


In [24]:
#Giving parameters that we want to tune
parameters = {
    'n_estimators': [50,100,500],
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [25]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5, n_jobs=-1)


In [26]:
search.fit(X_train,y_train)


GridSearchCV(cv=5, estimator=BaggingClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_features': [0.1, 0.4, 0.7, 1.0],
                         'max_samples': [0.1, 0.4, 0.7, 1.0],
                         'n_estimators': [50, 100, 500]})

In [27]:
search.best_params_

{'bootstrap': True,
 'max_features': 0.7,
 'max_samples': 1.0,
 'n_estimators': 500}

In [28]:
search.best_score_

np.float64(0.976375)